<a href="https://colab.research.google.com/github/wooihaw/practical_ai/blob/main/Handson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on 5A
## This is a 'Hello World" example for convolutional neural network (CNN)
### In this example, we will build and train a CNN to classify images of handwritten digit

In [ ]:
# Create a clone of practical_ai repo
!cd /content/
!git clone https://github.com/wooihaw/practical_ai.git

In [ ]:
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
# Import the relevant Python modules
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the dataset and split data into training and test sets
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# Display an handwritten digit in the traijning set (change m to display another digit)
m = 111
plt.axis(False)
plt.title(f'Label: {training_labels[m]}')
plt.imshow(training_images[m], cmap='gray')
plt.show()

In [ ]:
# Preprocessing the data
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images / 255.0

In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# model fitting (train the model using the training set)
history = model.fit(training_images, training_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=64)

In [ ]:
# Plot the performance of the model
plt.plot(history.history['accuracy'], 'b', label='train')
plt.plot(history.history['val_accuracy'], 'g', label='test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc='best')
plt.show()

plt.plot(history.history['loss'], 'b', label='train')
plt.plot(history.history['val_loss'], 'g', label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# Evaluate the performance of the trained model using the test set
score = model.evaluate(test_images, test_labels)
print(f'Loss: {score[0]:.4f}, accuracy: {score[1]:.4f}')

In [ ]:
# Predict the label of a test image (enter an index between 0 and 9999)
try:
  index = int(input("Enter an index between 0 and 9999: "))
  assert 0 <= index < 10000, "The index should be between 0 and 9999"
except Exception as e:
  print("Error: ", e)
else:
  x = np.expand_dims(test_images[index], axis=0)
  classes = model.predict(x)
  print(f'Predicted label: {classes.argmax()}')

  # Display the test image and show the actual label
  plt.axis(False)
  plt.title(f'Actual label: {test_labels[index]}')
  plt.imshow(test_images[index].reshape(28, 28) * 255, cmap='gray')
  plt.show()

# Hands-on 5B
## Classification of three animals (from scratch)
### In this example, we are going to train a CNN model from scratch for the classfication of three animals

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Extract dataset from Google Drive
!cd /content/
!tar -xf /content/drive/MyDrive/three_animals.tar.gz > /dev/null && echo "A total of $(find /content/three_animals/ -type f | wc -l) files have been successfully extracted."

In [ ]:
# Load modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Setup training images and testing images
train_dir = 'three_animals/train'
test_dir = 'three_animals/validate/'

train_datagen = ImageDataGenerator(
                    rescale=1/255.,
                    rotation_range=30,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical',
                    shuffle=False)

In [ ]:
# Construct CNN
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

# Compile and train model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(train_generator,
                    epochs=5,
                    validation_data=test_generator,
                    verbose=1)

In [ ]:
# Plot model accuracy and lose
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

plt.plot(np.arange(1, len(acc) + 1), acc, 'b', label='accuracy')
plt.plot(np.arange(1, len(val_acc) + 1), val_acc, 'g', label='val_accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(1, len(acc) + 1))
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
# Show the confusion matrix
pred = model.predict(test_generator)
y_pred = np.argmax(pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
plt.style.use('default')
fig = plt.figure()
ax = plt.gca()
im = ax.matshow(cm)
for i, j in enumerate(cm.ravel()):
    ax.text(i%3, i//3, f'{j}', color='w', size='large', weight='bold', ha='center')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticks(np.arange(3))
ax.set_xticklabels(list(train_generator.class_indices))
ax.set_yticks(np.arange(3))
ax.set_yticklabels(list(train_generator.class_indices))
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
plt.show()

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model1.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.weights.h5")
print("Saved model to disk")

from json import dump
with open('class_indices1.json', 'w') as f:
  dump(train_generator.class_indices, f)

In [ ]:
# Test the trained model on a new image
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from json import load
import numpy as np

# load json and create model
json_file = open('model1.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model1.weights.h5")

# load class indices
with open('class_indices1.json', 'r') as f:
    class_indices = load(f)
print("Model loaded from disk")
map2class = {class_indices[k]:k for k in class_indices}

In [ ]:
# Display the test image and show the predicted class
file = 'practical_ai/animal01.jpg'
img = load_img(file, target_size=(150, 150))
x = img_to_array(img)/255.
x = np.expand_dims(x, axis=0)
classes = model.predict(x)
for i, k in enumerate(class_indices):
  print(f'{k:10}: {classes[0, i]: .3f}')

plt.axis(False)
plt.imshow(load_img(file))
plt.title(f'Predicted as {map2class[classes.argmax()]}', y=0, pad=-10, verticalalignment="top")
plt.show()

# Hands-on 5C
## Classification of three animals (with transfer learning)
### In this example, we are going to train a CNN model with transfer learning for the classfication of three animals

In [ ]:
# Load modules
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

# Setup training images and testing images
train_dir = 'three_animals/train'
test_dir = 'three_animals/validate/'

train_datagen = ImageDataGenerator(
                    rescale=1/255.,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    batch_size=32,
                    target_size=(150,150),
                    class_mode='categorical',
                    shuffle=False)

In [ ]:
# Import the inception mode, do not include the fully-connected layer at the top as the last layer of the network
base_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Make all the layers in the pre-trained model non-trainable
for layer in base_model.layers:
    layer.trainable = False

last_layer = base_model.get_layer('mixed7')
last_output = last_layer.output

# Applies average pooling on the spatial dimensions until each spatial dimension is one
x = GlobalAveragePooling2D()(last_output)
# Add a fully connected layer with 1024 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.2
x = Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = Dense(3, activation='softmax')(x)

model = Model(base_model.input, x)

model.compile(optimizer ='rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

# Calculate the number of layers
num_layers = len(model.layers)

# Calculate the total number of parameters
total_params = model.count_params()

print(f'Number of layers: {num_layers}')
print(f'Total parameters to learn: {total_params}')

history = model.fit(train_generator,
                    epochs=3,
                    validation_data=test_generator,
                    verbose=1)

In [ ]:
# Plot model accuracy and lose
acc = history.history['acc']
loss = history.history['loss']
val_acc = history.history['val_acc']
val_loss = history.history['val_loss']

plt.plot(np.arange(1, len(acc) + 1), acc, 'b', label='accuracy')
plt.plot(np.arange(1, len(val_acc) + 1), val_acc, 'g', label='val_accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(1, len(acc) + 1))
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend(loc=0)
plt.show()

In [ ]:
# Show the confusion matrix
pred = model.predict(test_generator)
y_pred = np.argmax(pred, axis=1)
cm = confusion_matrix(test_generator.classes, y_pred)
plt.style.use('default')
fig = plt.figure()
ax = plt.gca()
im = ax.matshow(cm)
for i, j in enumerate(cm.ravel()):
    ax.text(i%3, i//3, f'{j}', color='w', size='large', weight='bold', ha='center')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticks(np.arange(3))
ax.set_xticklabels(list(train_generator.class_indices))
ax.set_yticks(np.arange(3))
ax.set_yticklabels(list(train_generator.class_indices))
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
plt.show()

In [ ]:
# List files that were missclassified
p = test_generator.classes != y_pred
file_errors = [test_generator.filenames[i] for i, j in enumerate(p) if j]
print(file_errors)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model2.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model2.weights.h5")
print("Saved model to disk")

from json import dump
with open('class_indices2.json', 'w') as f:
  dump(train_generator.class_indices, f)

In [ ]:
# Test the trained model on a new image
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from json import load
import numpy as np

# load json and create model
json_file = open('model2.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model2.weights.h5")

# load class indices
with open('class_indices2.json', 'r') as f:
    class_indices = load(f)
print("Model loaded from disk")
map2class = {class_indices[k]:k for k in class_indices}

In [ ]:
# Display the test image and show the predicted class
file = 'practical_ai/animal01.jpg'
img = load_img(file, target_size=(150, 150))
x = img_to_array(img)/255.
x = np.expand_dims(x, axis=0)
classes = model.predict(x)
for i, k in enumerate(class_indices):
  print(f'{k:10}: {classes[0, i]: .3f}')

plt.axis(False)
plt.imshow(load_img(file))
plt.title(f'Predicted as {map2class[classes.argmax()]}', y=0, pad=-10, verticalalignment="top")
plt.show()